In [40]:
import pandas as pd
from functions import assembleDf, epochElo, epochsElo, epochG, epochsG,PlayerElo
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [3]:
singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_5709/3293468424.py:1: DtypeWarning: Columns (3,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'],


In [4]:
playerClasses, ratingsHistory = epochsElo(matches)

In [93]:
 in [key[0] for key in ratingsHistory.keys()]

False

In [5]:
ratingsHistory_df = assembleDf(ratingsHistory)

In [6]:
ratingsHistory_df

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1878-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1879-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1880-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1881-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1882-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-05,NaN,NaN,1487.425524,1585.689393,NaN,NaN,NaN,NaN,NaN,1485.277608,...,NaN,NaN,1491.681934,NaN,1491.945372,1441.945582,1480.327348,1497.078707,NaN,1468.298904
2019-06-05,NaN,NaN,1473.301284,1580.784232,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1446.916587,NaN,NaN,NaN,1443.751284,1475.091242,NaN,NaN,1450.854843
2020-06-04,NaN,NaN,1489.892882,1574.674949,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1466.854386,NaN,NaN,1430.651881
2021-06-04,NaN,NaN,1482.284277,1564.924107,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ratingsHistory_df.columns.nunique(),ratingsHistory_df.columns.shape

(32316, (32316,))

In [8]:
# fill in missing values with last seen rating.  
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)

In [47]:
ratingsHistory_df.index

DatetimeIndex(['1877-07-09', '1878-07-09', '1879-07-09', '1880-07-08',
               '1881-07-08', '1882-07-08', '1883-07-08', '1884-07-07',
               '1885-07-07', '1886-07-07',
               ...
               '2013-06-06', '2014-06-06', '2015-06-06', '2016-06-05',
               '2017-06-05', '2018-06-05', '2019-06-05', '2020-06-04',
               '2021-06-04', '2022-06-04'],
              dtype='datetime64[ns]', length=146, freq=None)

In [41]:
def get_recent_rating_wp(ratingsHistory_df,tourney_date,winner_id,loser_id):
    if not type(tourney_date) == pd._libs.tslibs.timestamps.Timestamp:
        tourney_date = datetime.strptime(tourney_date, '%Y-%m-%d').date()

    timestamp = max(ratingsHistory_df.index[ratingsHistory_df.index<=tourney_date])
    winner_rating, loser_rating = ratingsHistory_df.loc[timestamp,[winner_id,loser_id]]
    wp = 1/(1 + 10**((loser_rating-winner_rating)/400))
    return winner_rating, loser_rating, wp

In [49]:
get_recent_rating_wp(ratingsHistory_df,pd.Timestamp('1877-07-09T00'),131867,131879)

(1500.0, 1500.0, 0.5)

In [10]:
padRow = pd.DataFrame({col: 1500.0 for col in ratingsHistory_df.columns}, index = [pd.Timestamp('1877-07-09T00')])
padRow

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0


In [11]:
ratingsHistory_df.columns.shape

(32316,)

In [12]:
ratingsHistory_df.shape, padRow.shape

((145, 32316), (1, 32316))

In [13]:
padRow.index

DatetimeIndex(['1877-07-09'], dtype='datetime64[ns]', freq=None)

In [14]:
ratingsHistory_df.shape

(145, 32316)

In [15]:
ratingsHistory_df = pd.concat([padRow,ratingsHistory_df],axis=0)

In [16]:
ratingsHistory_df

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1878-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1879-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1880-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1881-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-05,1500.00000,1502.609587,1487.425524,1585.689393,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1453.542135,1491.681934,1500.00000,1491.945372,1441.945582,1480.327348,1497.078707,1490.832406,1468.298904
2019-06-05,1500.00000,1502.609587,1473.301284,1580.784232,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1475.091242,1497.078707,1490.832406,1450.854843
2020-06-04,1500.00000,1502.609587,1489.892882,1574.674949,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881
2021-06-04,1500.00000,1502.609587,1482.284277,1564.924107,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881


In [17]:
matches.head(1)

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540


In [18]:
get_recent_rating_wp(ratingsHistory_df,'1877-07-09',113987,114149)

(1500.0, 1500.0, 0.5)

In [19]:
get_recent_rating_wp(ratingsHistory_df,'1878-07-09',113987,114149)

(1532.0, 1508.0, 0.5344839447268315)

In [20]:
matches

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
876973,2022-12-26,207987,208519,2022-M-ITF-TUN-63A-2022
876974,2022-12-26,144932,209977,2022-M-ITF-TUN-63A-2022
876975,2022-12-26,209955,209942,2022-M-ITF-TUN-63A-2022
876976,2022-12-26,209079,210696,2022-M-ITF-TUN-63A-2022


In [21]:
matches

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
876973,2022-12-26,207987,208519,2022-M-ITF-TUN-63A-2022
876974,2022-12-26,144932,209977,2022-M-ITF-TUN-63A-2022
876975,2022-12-26,209955,209942,2022-M-ITF-TUN-63A-2022
876976,2022-12-26,209079,210696,2022-M-ITF-TUN-63A-2022


In [22]:
singles_matches.shape,matches.shape

((876978, 49), (876978, 4))

In [23]:
ratingsHistory_df

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1878-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1879-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1880-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1881-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-05,1500.00000,1502.609587,1487.425524,1585.689393,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1453.542135,1491.681934,1500.00000,1491.945372,1441.945582,1480.327348,1497.078707,1490.832406,1468.298904
2019-06-05,1500.00000,1502.609587,1473.301284,1580.784232,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1475.091242,1497.078707,1490.832406,1450.854843
2020-06-04,1500.00000,1502.609587,1489.892882,1574.674949,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881
2021-06-04,1500.00000,1502.609587,1482.284277,1564.924107,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881


In [89]:
210082 in ratingsHistory_df.columns.values

False

In [85]:
113987 in matches[['winner_id','loser_id']]['winner_id']

True

In [96]:
matches.iloc[833942,:]

tourney_date        2021-06-07 00:00:00
winner_id                        210082
loser_id                         126152
tourney_id      2021-M-ITF-USA-08A-2021
Name: 833942, dtype: object

In [94]:
matches[['winner_id','loser_id']].loc[matches['winner_id']==210082,'winner_id']

833942    210082
Name: winner_id, dtype: int64

In [80]:
we_le_pw = pd.DataFrame(matches.apply(lambda x:
                           get_recent_rating_wp(
                               ratingsHistory_df, 
                               x['tourney_date'], 
                               x['winner_id'], 
                               x['loser_id']),
                            axis=1),columns = ['winner_elo','loser_elo','predicted_wins'])

KeyError: '[210082] not in index'

In [62]:
singles_matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1877-540,Wimbledon,Grass,32,G,1877-07-09,19,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1877-540,Wimbledon,Grass,32,G,1877-07-09,21,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1877-540,Wimbledon,Grass,32,G,1877-07-09,20,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1877-540,Wimbledon,Grass,32,G,1877-07-09,18,113963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1877-540,Wimbledon,Grass,32,G,1877-07-09,17,113999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876973,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,104,207987,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,716.0,31.0,966.0,12.0
876974,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,103,144932,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,799.0,23.0,1117.0,8.0
876975,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,102,209955,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876976,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,205,209079,NaN,JE,...,NaN,NaN,NaN,NaN,NaN,NaN,1125.0,7.0,1548.0,2.0


In [65]:
singles_r_wp = pd.concat([singles_matches,pd.DataFrame(matches.apply(lambda x: 
                           get_recent_rating_wp(
                               ratingsHistory_df, 
                               x['tourney_date'], 
                               x['winner_id'], 
                               x['loser_id']),
                            axis=1),columns = ['winner_elo','loser_elo','predicted_wins'])])

KeyError: "None of [Index([(113987,), (114149,)], dtype='object')] are in the [index]"

In [56]:
singles_matches[1:10].shape,matches[1:10].shape

((9, 49), (9, 4))

In [37]:
matches.dtypes

tourney_date    datetime64[ns]
winner_id                int64
loser_id                 int64
tourney_id              object
dtype: object

In [58]:
testDf

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,"(winner_elo, loser_elo, predicted_wins)"
1,1877-540,Wimbledon,Grass,32,G,1877-07-09,21,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
2,1877-540,Wimbledon,Grass,32,G,1877-07-09,20,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
3,1877-540,Wimbledon,Grass,32,G,1877-07-09,18,113963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
4,1877-540,Wimbledon,Grass,32,G,1877-07-09,17,113999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
5,1877-540,Wimbledon,Grass,32,G,1877-07-09,16,114158,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
6,1877-540,Wimbledon,Grass,32,G,1877-07-09,15,113999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
7,1877-540,Wimbledon,Grass,32,G,1877-07-09,13,114149,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
8,1877-540,Wimbledon,Grass,32,G,1877-07-09,12,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"
9,1877-540,Wimbledon,Grass,32,G,1877-07-09,11,113963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(1500.0, 1500.0, 0.5)"


In [59]:
matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)

1    (1500.0, 1500.0, 0.5)
2    (1500.0, 1500.0, 0.5)
3    (1500.0, 1500.0, 0.5)
4    (1500.0, 1500.0, 0.5)
5    (1500.0, 1500.0, 0.5)
6    (1500.0, 1500.0, 0.5)
7    (1500.0, 1500.0, 0.5)
8    (1500.0, 1500.0, 0.5)
9    (1500.0, 1500.0, 0.5)
dtype: object

In [57]:
testDf = singles_matches[1:10]
testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)


/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_5709/848510049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(


In [28]:
singles_matches[['winner_elo','loser_elo','predicted_wins']] = matches[['tourney_date','winner_id','loser_id']].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=1)

KeyboardInterrupt: 

In [ ]:
ratingsHistory_df